BDLE 2021

date du document  :  3/12/2021

# TP : accès parallèle, traitement sur des partitions

exemple d'accès en parallèle aux données du site Eau de France.
[Données de piezométrie](https://hubeau.eaufrance.fr/page/api-piezometrie) 

## Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [1]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/essai"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [2]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 54.3 MB/s 
installé


Démarrer la session spark

In [3]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [4]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1644507947822


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [5]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [6]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    for r in tabRequetes:
        if len(r.strip()) > 2:
          derniere = spark.sql(r)
    return display(derniere)
print("fonctions définies")

fonctions définies


Utiliaires : Chronomètres

In [7]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')
    
print("fonctions définies")

fonctions définies


## Accès aux données

In [8]:
import os
temp = "/temp/"
os.makedirs(temp, exist_ok=True)
os.listdir(temp)

[]

URL pour l'accès aux datasets

In [9]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [10]:
import os
from urllib import request

# acces online aux données
# https://www.egc.asso.fr/wp-content/uploads/data_Rostrenen.csv


def load_file(file):
  if(os.path.isfile( temp + file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/defi_EGC_2022/" + file
    print("downloading from URL: ", url, "save in : " + temp + file)
    request.urlretrieve(url , temp  + file)

load_file("points_eau.csv")
load_file("data_Rostrenen.csv")

# Liste des fichiers de IMDB
os.listdir(temp)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/points_eau.csv save in : /temp/points_eau.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/data_Rostrenen.csv save in : /temp/data_Rostrenen.csv


['data_Rostrenen.csv', 'points_eau.csv']

## Table Point : les points d'eau

In [11]:
point = spark.read.option("delimiter",";").option("header",True).csv(temp + 'points_eau.csv').repartition(4).persist()
point.createOrReplaceTempView("point")
display(point, 3)

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,00762X0004/S1,BSS000FHYM,0.860993857091503,49.6497369721205,76456,MOTTEVILLE,H9923020,L'Austreberthe à Saint-Paer,121AU01,Craie du Séno-Turonien du Bassin Parisien de l...
1,07223C0113/S,BSS001URLS,4.9003527411453,45.6473838620031,69273,CORBAS,None,None,521AK00,"NV3 absent, nom de l'entité NV2 : Formations f..."
2,00487X0015/S1,BSS000EECH,3.07279822338651,49.9035922409316,80413,HANCOURT,E6351408,Haute Somme à Ham,121BB01,Craie du Séno-Turonien du bassin versant de la...


Nombre de points d'eau

In [12]:
point.count()

18

## Requetes d'analyse

In [13]:
%%sql
select *
from Point
where nom_commune like '%P%'

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,07476X0029/S,BSS001VTUD,5.19017582950695,45.3650549559674,38300,PENOL,None,None,521AM00,"NV3 absent, nom de l'entité NV2 : Alluvions fl..."
1,01516X0004/S1,BSS000LETA,1.61497845782586,48.9815660459082,78484,PERDREAUVILLE,None,None,121AZ01,Craie du Séno-Turonien du Bassin Parisien du V...
2,06505X0080/FORC,BSS001REHG,4.76295450181247,46.1383028294756,69242,TAPONAS,None,None,507AD02,Sables pliocènes du Val de Saône
3,00755X0006/S1,BSS000FHCQ,0.419809838577633,49.5555270637694,76714,LES TROIS-PIERRES,G9103020,Lézarde à Montivilliers,121AU01,Craie du Séno-Turonien du Bassin Parisien de l...
4,02706X0074/S77-20,BSS000UTLD,6.9302772399532,48.4395463138954,88082,CELLES-SUR-PLAINE,None,None,143AK07,Grès d'Annweiler et Grès de Senones du Buntsan...


In [14]:
%%sql
select min(longitude)
from Point


,min(longitude)
0,-3.30717741173568


## Accès Open data online

Example d'URL : https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=07548X0009/F&sort=desc&size=10
les paramêtres de l'appel sont
*   *code_bss* : le code du point d'eau
*   *sort* : pour **trier** les données par date décroissante : sort=desc
*   *size* : le nombre de tuples retournés

In [15]:
import urllib.request
import json

# code_bss = "07548X0009/F"
code_bss = "01516X0004/S1"

urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&sort=desc&size=20"

with urllib.request.urlopen(urlEau) as response:
   encoding = response.info().get_content_charset('utf-8')
  # obj = json.loads(response.read().decode(encoding))
  # texte = json.dumps(obj)
   texte = response.read().decode(encoding)

jrdd = spark.sparkContext.parallelize([texte])
data1 = spark.read.json(jrdd)
# data1.printSchema()
data2 = data1.selectExpr("explode(data) as item").\
selectExpr("item.code_bss as code",\
           "item.profondeur_nappe as niveau",\
           "item.date_mesure as date",\
           "item.timestamp_mesure as time")
data3 = data2.orderBy(desc(data2.date))
data3.persist()
data3.printSchema()
display(data3)

root
 |-- code: string (nullable = true)
 |-- niveau: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: long (nullable = true)



,code,niveau,date,time
0,01516X0004/S1,20.10,2022-02-07,1644192000000
1,01516X0004/S1,20.09,2022-02-06,1644163200000
2,01516X0004/S1,20.11,2022-02-05,1644019200000
3,01516X0004/S1,20.09,2022-02-04,1643972400000
4,01516X0004/S1,20.10,2022-02-03,1643864400000
5,01516X0004/S1,20.11,2022-02-02,1643760000000
6,01516X0004/S1,20.10,2022-02-01,1643731200000
7,01516X0004/S1,20.10,2022-01-31,1643594400000
8,01516X0004/S1,20.11,2022-01-30,1643558400000
9,01516X0004/S1,20.12,2022-01-29,1643425200000


# Exercice 1 : Accès en parallèle aux mesures d'eau

## Question 1 : Accès parallèle

Pour toutes les stations (code_bss) de la table Point. Accéder en parallèle au site pour récupérer les 10 dernières mesures de niveau d'eau.
Le résultat est un seul dataframe Mesure(code,niveau,date,time) contenant toutes les mesures de tous les points d'eau. Rendre persistant le dataframe Mesure.

In [50]:
def getNiveau(code_bss):
  urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&sort=desc&size=10"
  with urllib.request.urlopen(urlEau) as response:
    encoding = response.info().get_content_charset('utf-8')
    obj = json.loads(response.read().decode(encoding))
  return obj['data']

typeRetour = ArrayType(MapType(StringType(), StringType()))
getNiveauUDF = udf(getNiveau, typeRetour)

r1 = point.withColumn("Niveaux", getNiveauUDF(col("code_bss"))).select('code_bss', explode('Niveaux').alias('Niveau'))

Mesure = r1.rdd.map(lambda x: (x.code_bss, float(x.Niveau["profondeur_nappe"]), x.Niveau["date_mesure"], float(x.Niveau["timestamp_mesure"])))\
.toDF(["code","niveau","date","time"]).persist()

display(Mesure)

,code,niveau,date,time
0,00762X0004/S1,32.93,2022-02-02,1.643760e+12
1,00762X0004/S1,32.91,2022-02-01,1.643699e+12
2,00762X0004/S1,32.88,2022-01-31,1.643598e+12
3,00762X0004/S1,32.89,2022-01-30,1.643580e+12
4,00762X0004/S1,32.89,2022-01-29,1.643429e+12
...,...,...,...,...
95,06505X0080/FORC,10.77,2022-01-26,1.643159e+12
96,06505X0080/FORC,10.74,2022-01-25,1.643069e+12
97,06505X0080/FORC,10.71,2022-01-24,1.642982e+12
98,06505X0080/FORC,10.68,2022-01-23,1.642900e+12


## Question 2 : degré de parallélisme

En faisant varier le nombre de partitions de 1 à 2, comparer la durée pour générer le dataframe Mesure.
Calculer le rapport des durées.

In [51]:
import time

start_time = time.time()
temps1Partition = Mesure.repartition(1).persist()
temps1Partition.count()
exec_time = time.time() - start_time
print("Temps partition 1 : --- %s seconds ---" % (exec_time))


start_time = time.time()
temps2Partitions = Mesure.repartition(2).persist()
temps2Partitions.count()
exec_time2 = time.time() - start_time
print("Temps partition 2 : --- %s seconds ---" % (exec_time2))

print("\nRapport des durées :", exec_time2/exec_time)

Temps partition 1 : --- 4.3180108070373535 seconds ---
Temps partition 2 : --- 0.43598151206970215 seconds ---

Rapport des durées : 0.10096813823604926


# Exercice 2 : Traitement avec des partitions

On fournit des fonctions pour afficher le contenu des partitions d'un dataframe


#### Fonctions showPartitions et showPartitionSize
 
* showPartitions : affiche les _n_ premiers éléments de chaque partition
* showPartitionSize : affiche le nombre d'éléments dans chaque partition

In [37]:
# fonction auxilliaire
def partSize(partID, iterateur):
  c=0
  suivant = next(iterateur, None)
  while suivant is not None :
    c+=1
    suivant = next(iterateur, None)
  return [(partID, c)]


def showPartitionSize(df):  
  t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize)
  for (partID, nbElt) in t.collect():
    print("partition", partID, ":", nbElt, "éléments")
  print()


def showPartitions(df, N=5):
  size = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).collectAsMap()
  
  def topN(partID, iterateur):
    c=0
    head=[]
    suivant = next(iterateur, None)
    while suivant is not None and c < N :
      c+=1
      head.append(suivant)
      suivant = next(iterateur, None)
    return [(partID, head)]  
  t = df.rdd.mapPartitionsWithIndex(topN)
  for (partID, head) in t.collect():
    print("Partition", partID, ",", size[partID], "éléments")
    for row in head:
        print(row)
    print()
    
print('showPartitions et showPartitionSize définies')

showPartitions et showPartitionSize définies


Afficher la taille de chaque partition

In [38]:
showPartitionSize(data3)

partition 0 : 5 éléments
partition 1 : 5 éléments
partition 2 : 5 éléments
partition 3 : 5 éléments



Afficher les premiers éléments de chaque partition

In [39]:
showPartitions(data3)

Partition 0 , 5 éléments
Row(code='01516X0004/S1', niveau=20.1, date='2022-02-07', time=1644192000000)
Row(code='01516X0004/S1', niveau=20.09, date='2022-02-06', time=1644163200000)
Row(code='01516X0004/S1', niveau=20.11, date='2022-02-05', time=1644019200000)
Row(code='01516X0004/S1', niveau=20.09, date='2022-02-04', time=1643972400000)
Row(code='01516X0004/S1', niveau=20.1, date='2022-02-03', time=1643864400000)

Partition 1 , 5 éléments
Row(code='01516X0004/S1', niveau=20.11, date='2022-02-02', time=1643760000000)
Row(code='01516X0004/S1', niveau=20.1, date='2022-02-01', time=1643731200000)
Row(code='01516X0004/S1', niveau=20.1, date='2022-01-31', time=1643594400000)
Row(code='01516X0004/S1', niveau=20.11, date='2022-01-30', time=1643558400000)
Row(code='01516X0004/S1', niveau=20.12, date='2022-01-29', time=1643425200000)

Partition 2 , 5 éléments
Row(code='01516X0004/S1', niveau=20.12, date='2022-01-28', time=1643328000000)
Row(code='01516X0004/S1', niveau=20.12, date='2022-01-27',

## Question 3 : Fonction niveauMoyen

En vous inspirant des fonctions topN et showPartitions définir une fonction qui détermine le niveau moyen dans chaque partition. Le résultat est (numP, code, niveauMoyen) avec numP étant le numéro de partition.

In [52]:
import builtins
def niveauMoyen(df):
  size = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).collectAsMap()
  
  def topN(partID, iterateur):
    head={}

    suivant = next(iterateur, None)
    while suivant is not None :
      if suivant["code"] in head :
        head[suivant["code"]].append(suivant["niveau"])
      else:
        head[suivant["code"]] = [suivant["niveau"]]

      suivant = next(iterateur, None)

    res = [Row(code=k, moy=builtins.sum(v)/len(v)) for k,v in head.items()]
    return [(partID, res)]  
    
  t = df.rdd.mapPartitionsWithIndex(topN)
  for (partID, head) in t.collect():
    print("Partition", partID, ",", size[partID], "éléments")
    for row in head:
        print("Code",row["code"],": Moyenne =" , row["moy"])
    print()
    
print('niveauMoyen définie')

niveauMoyen définie


In [53]:
niveauMoyen(Mesure)

Partition 0 , 50 éléments
Code 00762X0004/S1 : Moyenne = 32.894
Code 07223C0113/S : Moyenne = 6.470999999999999
Code 00487X0015/S1 : Moyenne = 30.663
Code 07476X0029/S : Moyenne = 28.108999999999998
Code 01584X0023/LV3 : Moyenne = 11.587

Partition 1 , 40 éléments
Code 01258X0020/S1 : Moyenne = 13.777000000000001
Code 04398X0002/SONDAG : Moyenne = 19.668
Code 01516X0004/S1 : Moyenne = 20.103
Code 02206X0022/S1 : Moyenne = 39.729

Partition 2 , 40 éléments
Code 06505X0080/FORC : Moyenne = 10.775
Code 02603X0009/S1 : Moyenne = 12.565000000000001
Code 03124X0088/F : Moyenne = 11.933
Code 00755X0006/S1 : Moyenne = 83.34700000000001

Partition 3 , 50 éléments
Code 02706X0074/S77-20 : Moyenne = 15.898
Code 00471X0095/PZ2013 : Moyenne = 9.03
Code 02267X0030/S1 : Moyenne = 1.2049999999999996
Code 00766X0004/S1 : Moyenne = 68.08999999999999
Code 02648X0020/S1 : Moyenne = 6.8629999999999995



Invoquez la fonction que vous aurez définie

In [ ]:
# exemple d'invocation de la fonction partsize définie ci-dessus pour définir un dataframe.
pointPartSize = point.rdd.mapPartitionsWithIndex(partSize).toDF(["numP", "size"])
display(pointPartSize)

,numP,size
0,0,5
1,1,4
2,2,4
3,3,5


## Question 4 : NiveauTotalJour

Definir MesureParJour qui est un partitionnement de Mesure par date. 
En vous inspirant de la fonctions showPartitions,
définir une fonction qui détermine le niveau total par jour dans chaque partition. Le résultat est (numP, date, niveauTotal) 

In [54]:
def niveauTotalJour(df):
  size = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).collectAsMap()
  
  def topN(partID, iterateur):
    head={}

    suivant = next(iterateur, None)
    while suivant is not None :
      if suivant["date"] in head :
        head[suivant["date"]].append(suivant["niveau"])
      else:
        head[suivant["date"]] = [suivant["niveau"]]

      suivant = next(iterateur, None)

    res = [Row(date=k, sum=builtins.sum(v)) for k,v in head.items()]
    return [(partID, res)]  
    
  t = df.rdd.mapPartitionsWithIndex(topN)
  for (partID, head) in t.collect():
    print("Partition", partID, ",", size[partID], "éléments")
    for row in head:
        print(row["date"],": Niveau total =" , row["sum"])
    print()
    
print('niveauTotalJour définie')

niveauTotalJour définie


In [55]:
niveauTotalJour(Mesure)

Partition 0 , 50 éléments
2022-02-02 : Niveau total = 75.18
2022-02-01 : Niveau total = 75.14
2022-01-31 : Niveau total = 109.63
2022-01-30 : Niveau total = 109.65
2022-01-29 : Niveau total = 109.66000000000001
2022-01-28 : Niveau total = 67.47999999999999
2022-01-27 : Niveau total = 67.47
2022-01-26 : Niveau total = 67.47
2022-01-25 : Niveau total = 67.48
2022-01-24 : Niveau total = 67.46
2022-01-23 : Niveau total = 34.6
2022-01-22 : Niveau total = 34.61
2022-02-07 : Niveau total = 42.29
2022-02-06 : Niveau total = 42.29
2022-02-05 : Niveau total = 42.31
2022-02-04 : Niveau total = 42.269999999999996
2022-02-03 : Niveau total = 42.25

Partition 1 , 40 éléments
2022-02-07 : Niveau total = 73.68
2022-02-06 : Niveau total = 73.62
2022-02-05 : Niveau total = 73.71000000000001
2022-02-04 : Niveau total = 73.57
2022-02-03 : Niveau total = 73.59
2022-02-02 : Niveau total = 94.55
2022-02-01 : Niveau total = 94.24000000000001
2022-01-31 : Niveau total = 93.87
2022-01-30 : Niveau total = 93.62
